In [37]:
import pandas as pd
import re
import numpy as np
import importlib
from sklearn.ensemble import RandomForestRegressor
import ProcessingFunctions as pf
importlib.reload(pf)

<module 'ProcessingFunctions' from '/Users/DewiGould/Desktop/Kaggle/SpaceTitanic/ProcessingFunctions.py'>

In [39]:
df_test = pd.read_csv('spaceship-titanic-data/test.csv')

df_test = pf.replace_null_with_U(df_test,'PassengerId')
df_test = pf.split_passenger_id_column(df_test)



notnull = df_test[~df_test['HomePlanet'].isnull()]
dictionary_homeplanets = notnull.set_index('gggg')['HomePlanet'].to_dict()
df_test['HomePlanet'] = df_test['gggg'].apply(pf.update_homeplanet,args=(dictionary_homeplanets,))




df_test = pf.replace_nulls_with_means(df_test)
df_test = pf.replace_nulls_with_Us(df_test)
df_test = pf.split_cabin_column(df_test)

# For now, let's just try making all binary columns OneHotEncoders to include the Null values.
categorical_columns = ['HomePlanet','Destination','CryoSleep','VIP','deck','side']
df_test = pf.one_hot_encode_columns(df_test,categorical_columns)

# StandardScaler.
cols_to_scale = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
df_test,empty = pf.scale_numerical_columns(df_test,cols_to_scale, scaler)

In [90]:
# remove several columns for now, to make life easier.
X_dataset = df.drop(columns=['PassengerId','gggg','pp','number','Name'])
training = X_dataset.sample(n=5000)
X_train = training.drop(columns=['Transported'])
y_train = training.loc[:,'Transported'].astype(int)
X_test = X_dataset[~X_dataset.index.isin(X_train.index)].drop(columns=['Transported']) #complement of training set.
y_test = X_dataset[~X_dataset.index.isin(X_train.index)].loc[:,'Transported'].astype(int)

In [91]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

y_pred = [round(i) for i in y_pred]


In [92]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[1481,  351],
       [ 495, 1366]])

In [46]:
# remove several columns for now, to make life easier.
X_train = df.drop(columns=['Transported','PassengerId','gggg','pp','number','Name'])
y_train = df.loc[:,'Transported']
X_test = df_test.drop(columns=['PassengerId','gggg','pp','number','Name'])


regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

y_pred = [round(i) for i in y_pred]

X_test['Transported'] = y_pred

In [47]:
df_test_submit = pd.read_csv('spaceship-titanic-data/test.csv')
df_combined = df_test_submit.combine_first(X_test[["Transported"]])
df_combined['Transported'] = df_combined['Transported'].fillna(1)
df_combined['Transported'] = df_combined['Transported'].astype(bool)
df_combined[['PassengerId','Transported']].to_csv('submission_homeplanet_nulls.csv',index=False)